In [3]:
import pandas as pd
import numpy as np

In [12]:

#Importing the Dataset
customer_url = 'https://drive.google.com/uc?id=1bu_--mo79VdUG9oin4ybfFGRUSXAe-WE'
customer_data=pd.read_csv(customer_url)

product_url = 'https://drive.google.com/uc?id=1IKuDizVapw-hyktwfpoAoaGtHtTNHfd0'
product_data=pd.read_csv(product_url)

transactions_url = 'https://drive.google.com/uc?id=1saEqdbBB-vuk2hxoAf4TzDEsykdKlzbF'
transactions_data=pd.read_csv(transactions_url)

In [13]:
customer_data.head()

,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15


In [16]:
product_data.head()

,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.30
1,P002,ActiveWear Smartwatch,Electronics,346.30
2,P003,ComfortLiving Biography,Books,44.12
3,P004,BookWorld Rug,Home Decor,95.69
4,P005,TechPro T-Shirt,Clothing,429.31


In [18]:
transactions_data.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68


In [20]:
missing_values = customer_data.isnull().sum()
print("Missing Values in Customer Table:\n", missing_values)

Missing Values in Customer Table:
 CustomerID      0
CustomerName    0
Region          0
SignupDate      0
dtype: int64


In [24]:
missing_values = transactions_data.isnull().sum()
print("Missing Values in Transactions Table:\n", missing_values)

Missing Values in Transactions Table:
 TransactionID      0
CustomerID         0
ProductID          0
TransactionDate    0
Quantity           0
TotalValue         0
Price              0
dtype: int64


In [30]:
customer_data['SignupDate'] = pd.to_datetime(customer_data['SignupDate'])
transactions_data['TransactionDate'] = pd.to_datetime(transactions_data['TransactionDate'])


In [32]:
merged_df = pd.merge(transactions_data, customer_data[['CustomerID', 'Region', 'SignupDate']], on='CustomerID', how='left')

In [34]:
# Total spend per customer: Sum of TotalValue for each customer
total_spend_per_customer = merged_df.groupby('CustomerID')['TotalValue'].sum().reset_index()
total_spend_per_customer.rename(columns={'TotalValue': 'TotalSpend'}, inplace=True)


In [36]:
# Average transaction value: Mean of TotalValue per transaction
avg_transaction_value = merged_df.groupby('CustomerID')['TotalValue'].mean().reset_index()
avg_transaction_value.rename(columns={'TotalValue': 'AvgTransactionValue'}, inplace=True)

In [38]:
transaction_frequency = merged_df.groupby('CustomerID')['TransactionID'].count().reset_index()
transaction_frequency.rename(columns={'TransactionID': 'TransactionFrequency'}, inplace=True)


In [42]:
# Product preferences: Count of different product categories purchased by each customer
product_preferences = merged_df.groupby('CustomerID')['ProductID'].nunique().reset_index()
product_preferences.rename(columns={'ProductID': 'ProductPreferences'}, inplace=True)

In [50]:
from datetime import datetime
recency_of_last_purchase = merged_df.groupby('CustomerID')['TransactionDate'].max().reset_index()
recency_of_last_purchase['Recency'] = (datetime.now() - recency_of_last_purchase['TransactionDate']).dt.days
recency_of_last_purchase = recency_of_last_purchase[['CustomerID', 'Recency']]


In [52]:
customer_features = pd.merge(total_spend_per_customer, avg_transaction_value, on='CustomerID', how='left')
customer_features = pd.merge(customer_features, transaction_frequency, on='CustomerID', how='left')
customer_features = pd.merge(customer_features, recency_of_last_purchase, on='CustomerID', how='left')
customer_features = pd.merge(customer_features, product_preferences, on='CustomerID', how='left')


In [54]:
customer_features

,CustomerID,TotalSpend,AvgTransactionValue,TransactionFrequency,Recency,ProductPreferences
0,C0001,3354.52,670.904000,5,86,5
1,C0002,1862.74,465.685000,4,55,4
2,C0003,2725.38,681.345000,4,155,4
3,C0004,5354.88,669.360000,8,35,8
4,C0005,2034.24,678.080000,3,84,3
...,...,...,...,...,...,...
194,C0196,4982.88,1245.720000,4,43,3
195,C0197,1928.65,642.883333,3,30,3
196,C0198,931.83,465.915000,2,114,2
197,C0199,1979.28,494.820000,4,93,4


In [56]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler()
features_to_normalize = ['TotalSpend', 'AvgTransactionValue', 'Recency']

customer_features[features_to_normalize] = scaler.fit_transform(customer_features[features_to_normalize])
print(customer_features.head())

  CustomerID  TotalSpend  AvgTransactionValue  TransactionFrequency   Recency  \
0      C0001    0.308942             0.474336                     5  0.155556   
1      C0002    0.168095             0.308940                     4  0.069444   
2      C0003    0.249541             0.482751                     4  0.347222   
3      C0004    0.497806             0.473092                     8  0.013889   
4      C0005    0.184287             0.480120                     3  0.150000   

   ProductPreferences  
0                   5  
1                   4  
2                   4  
3                   8  
4                   3  


In [64]:
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
features = ['TotalSpend', 'AvgTransactionValue', 'TransactionFrequency', 'Recency', 'ProductPreferences']
customer_data = customer_features[features].values
euclidean_sim = -euclidean_distances(customer_data)
euclidean_sim_df = pd.DataFrame(euclidean_sim, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])

first_20_customers = [f'C{i:04d}' for i in range(1, 21)]
euclidean_sim_df = euclidean_sim_df.loc[first_20_customers, first_20_customers]

def get_top_n_lookalikes(similarity_matrix, n=3):
    lookalikes = {}
    
    for customer_id in similarity_matrix.columns:
        customer_similarities = similarity_matrix[customer_id].drop(customer_id)
        top_n_similar = customer_similarities.nlargest(n)
        lookalikes[customer_id] = list(zip(top_n_similar.index, top_n_similar.values))

    return lookalikes
lookalikes_euclidean = get_top_n_lookalikes(euclidean_sim_df, n=3)
lookalike_data = []

for customer_id, lookalikes in lookalikes_euclidean.items():
    lookalike_ids = [lookalike for lookalike, _ in lookalikes]
    scores = [score for _, score in lookalikes]
    lookalike_data.append([customer_id] + [item for pair in zip(lookalike_ids, scores) for item in pair])
columns = ['cust_id', 'lookalike_id_1', 'score_1', 'lookalike_id_2', 'score_2', 'lookalike_id_3', 'score_3']
lookalike_df = pd.DataFrame(lookalike_data, columns=columns)
lookalike_df.to_csv('Lookalike.csv', index=False)
print(lookalike_df.head())


  cust_id lookalike_id_1   score_1 lookalike_id_2   score_2 lookalike_id_3  \
0   C0001          C0016 -0.100286          C0011 -0.165589          C0018   
1   C0002          C0010 -0.054946          C0003 -0.337645          C0006   
2   C0003          C0010 -0.323627          C0002 -0.337645          C0006   
3   C0004          C0017 -0.101439          C0013 -1.430360          C0012   
4   C0005          C0007 -0.251063          C0009 -0.330219          C0015   

    score_3  
0 -0.365504  
1 -0.549507  
2 -0.354955  
3 -2.236988  
4 -1.424149  
